In [6]:
import cv2
import dlib
import numpy as np
import pyautogui

# Initialize Dlib's face detector and shape predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("./archive/shape_predictor_68_face_landmarks.dat")

# Webcam capture
cap = cv2.VideoCapture(0)

screen_width, screen_height = pyautogui.size()


def midpoint(p1, p2):
    return int((p1.x + p2.x) / 2), int((p1.y + p2.y) / 2)


def get_eye_region(eye_points, landmarks):
    region = np.array(
        [(landmarks.part(point).x, landmarks.part(point).y) for point in eye_points],
        np.int32,
    )
    return region


def get_gaze_ratio(eye_region, frame, gray):
    mask = np.zeros_like(gray)
    cv2.fillPoly(mask, [eye_region], 255)

    eye = cv2.bitwise_and(frame, frame, mask=mask)

    min_x = np.min(eye_region[:, 0])
    max_x = np.max(eye_region[:, 0])
    min_y = np.min(eye_region[:, 1])
    max_y = np.max(eye_region[:, 1])

    gray_eye = gray[min_y:max_y, min_x:max_x]
    _, threshold_eye = cv2.threshold(gray_eye, 70, 255, cv2.THRESH_BINARY)

    height, width = threshold_eye.shape
    left_side_threshold = threshold_eye[0:height, 0 : int(width / 2)]
    left_side_white = cv2.countNonZero(left_side_threshold)

    right_side_threshold = threshold_eye[0:height, int(width / 2) : width]
    right_side_white = cv2.countNonZero(right_side_threshold)

    if left_side_white == 0:
        gaze_ratio = 1
    elif right_side_white == 0:
        gaze_ratio = 5
    else:
        gaze_ratio = left_side_white / right_side_white

    return gaze_ratio


while True:
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = detector(gray)
    for face in faces:
        landmarks = predictor(gray, face)

        # Left eye region (landmarks 36 to 41)
        left_eye_region = get_eye_region([36, 37, 38, 39, 40, 41], landmarks)
        gaze_ratio_left = get_gaze_ratio(left_eye_region, frame, gray)

        # Right eye region (landmarks 42 to 47)
        right_eye_region = get_eye_region([42, 43, 44, 45, 46, 47], landmarks)
        gaze_ratio_right = get_gaze_ratio(right_eye_region, frame, gray)

        gaze_ratio = (gaze_ratio_left + gaze_ratio_right) / 2

        # Map gaze ratio to screen coordinates
        if gaze_ratio < 1:  # Looking left
            screen_x = 0
        elif gaze_ratio > 1.5:  # Looking right
            screen_x = screen_width
        else:
            screen_x = screen_width / 2

        screen_y = screen_height / 2  # Assuming only horizontal tracking for now

        # Move mouse to the screen position (to simulate gaze)
        pyautogui.moveTo(screen_x, screen_y)

        # Draw left and right eye regions
        cv2.polylines(frame, [left_eye_region], True, (0, 255, 0), 2)
        cv2.polylines(frame, [right_eye_region], True, (0, 255, 0), 2)

    cv2.imshow("Gaze Tracker", frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 

: 